# Wstęp
W tej części projektu zajmiemy się budową i oceną różnych modeli uczenia maszynowego, które klasyfikują przepisy kulinarne na podstawie składników. Wykorzystamy zarówno tradycyjne metody klasyfikacji, takie jak regresja logistyczna, SVM, Random Forest, jak i głębokie sieci neuronowe z warstwami RNN i LSTM. Wszystkie modele będą trenowane na tych samych danych, co pozwoli na porównanie ich wydajności.

## Import biblitek

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from lime.lime_text import LimeTextExplainer
from sklearn.model_selection import StratifiedKFold
from yellowbrick.model_selection import LearningCurve
from yellowbrick.text.correlation import WordCorrelationPlot
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from wordcloud import WordCloud
from matplotlib import pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_validate, KFold
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from gensim.models import Word2Vec
from sklearn.base import BaseEstimator, TransformerMixin
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, GlobalAveragePooling1D, Bidirectional, LSTM

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python3 -m spacy download pl_core_news_lg

  Using cached https://github.com/explosion/spacy-models/releases/download/pl_core_news_lg-3.7.0/pl_core_news_lg-3.7.0-py3-none-any.whl (573.7 MB)
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
import pl_core_news_lg

In [ ]:
# Monotowanie dysku Googla
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Wczytujemy przetworzone dane

In [ ]:
# Skopiowanie pliku z danymi po przetworzeniu
!cp '/content/drive/My Drive/PK-DS-JZTW-dane/'df_after_processing.parquet /content/
df_after_processing = pd.read_parquet('/content/drive/MyDrive/PK-DS-JZTW-dane/df_after_processing.parquet')

In [ ]:
df_after_processing.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2555 entries, 0 to 2556
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   category               2555 non-null   object 
 1   recipe_title           2555 non-null   object 
 2   ingredients            2555 non-null   object 
 3   preparation_method     2555 non-null   object 
 4   rating                 2264 non-null   float64
 5   opinions_count         757 non-null    float64
 6   preparation_time       1786 non-null   float64
 7   vegetarian             1795 non-null   boolean
 8   processed_ingredients  2555 non-null   object 
dtypes: boolean(1), float64(3), object(5)
memory usage: 184.6+ KB


In [ ]:
df_after_processing.head()

,category,recipe_title,ingredients,preparation_method,rating,opinions_count,preparation_time,vegetarian,processed_ingredients
0,sniadanie,Jajka z rzeżuchą,10 jajek;4 łyżki majonezu;2 łyżeczki musztardy...,Jajka ugotować na twardo (6 minut od zagotowan...,NaN,NaN,NaN,<NA>,"jajek, majonezu, musztardy miodowej, sosu worc..."
1,sniadanie,Pasta z fasoli,1 puszka białej fasolki (400 g);150 g korzenia...,Fasolkę odcedzić na sitku. Selera obrać i pokr...,NaN,NaN,NaN,<NA>,"puszka białej fasolki ( g), korzenia selera, p..."
2,sniadanie,Jajka z majonezem truflowym,5 jajek;2 łyżki majonezu;ok. 2 łyżeczek pasty ...,"Jajka ugotować na twardo, obrać ze skorupek, p...",NaN,NaN,NaN,<NA>,"jajek, majonezu, ok pasty salsy truflowej, nat..."
3,sniadanie,Pan bagnat,"1 większa, okrągła lub podłużna bułka, np. cia...",Pieczywo przekroić w poziomie. Położyć na blac...,5.0,4.0,NaN,<NA>,"większa, okrągła podłużna bułka, np ciabatta, ..."
4,sniadanie,Tosty francuskie z solonym karmelem,4 kromki brioche lub chałki;1 duże jajko;5 łyż...,W głębokim talerzu roztrzepać (widelcem lub ró...,5.0,2.0,NaN,<NA>,"kromki brioche chałki, duże jajko, mleka, cukr..."


## Przygotowanie danych do trenowania modeli, dzielimy je na zestawy treningowe i testowe

In [ ]:
# Definiuję zbiory danych niezależnych (cechy) i zależnych (etykiety)
X = df_after_processing['ingredients']
y = df_after_processing['category']

# Dzielę zbiory dane na zestaw treningowy i testowy (który odkładamy do walidacji)
X_train_global, X_test_global, y_train_global, y_test_global = train_test_split(X,
                                                                                y,
                                                                                test_size=0.2,
                                                                                random_state=42,
                                                                                stratify=y) # parametr ten zapewnia proporcje klas w zbiorze ternigowym i testowym będą takie same
X = X_train_global
y = y_train_global

Dane testowe będą zarezerwowane do oceny (walidacji) w naszym przypadku jest to 20% wydajności modelu po jego wytrenowaniu, a 80% danych będzie użyta do trenowania modelu. Dzięki temu podziałowi możemy monitorować czy model dobrze się uczy generalizować czy może jest nadmiernie dopasowany (overfitting) do danych treningowych. Zbiór walidacyjny pozwala również na ocenę jak dobrze model będzie działał na nowych, niewidzianych wcześniej danych.

## Model NLP z zatosowanie kolumny ingredients

Model NLP (Natural Language Processing) czyli przetwarzanie języka naturalnego, w tym przypadku do klasyfikacji przepisów kulinarnych na podstawie składników, użyjemy takich modeli jak Logistic Regresion, SVM, RandonForestClassifier, GradientBoostingClassifier.

In [ ]:
# Definiowanie pipeline dla regresji logistycznej
pipe_lr = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('lr', LogisticRegression())
])

# Grid search dla regresji logistycznej
param_grid_lr = {
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__min_df': [1, 2, 3],
    'lr__C': [0.01, 0.1, 1, 10, 100]
}

grid_search_lr = GridSearchCV(pipe_lr, param_grid_lr, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_lr.fit(X_train_global, y_train_global)

# Ocena modelu na danych treningowych i testowych
print("Ocena modelu regresji logistycznej na danych treningowych:")
print(classification_report(y_train_global, grid_search_lr.predict(X_train_global)))
print("Ocena modelu regresji logistycznej na danych testowych:")
print(classification_report(y_test_global, grid_search_lr.predict(X_test_global)))

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Ocena modelu regresji logistycznej na danych treningowych:
              precision    recall  f1-score   support

     kolacja       0.78      0.77      0.77       786
       obiad       0.83      0.86      0.84       787
   sniadanie       0.86      0.82      0.84       471

    accuracy                           0.82      2044
   macro avg       0.82      0.82      0.82      2044
weighted avg       0.82      0.82      0.82      2044

Ocena modelu regresji logistycznej na danych testowych:
              precision    recall  f1-score   support

     kolacja       0.66      0.69      0.68       196
       obiad       0.78      0.79      0.78       197
   sniadanie       0.78      0.70      0.74       118

    accuracy                           0.73       511
   macro avg       0.74      0.73      0.73       511
weighted avg       0.73      0.73      0.73       511



In [ ]:
# Pipeline dla SVM
pipe_svm = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('svm', SVC())
])

# Grid search dla SVM
param_grid_svm = {
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__min_df': [1, 2, 3],
    'svm__C': [0.1, 1, 10],
    'svm__kernel': ['linear', 'rbf']
}

grid_search_svm = GridSearchCV(pipe_svm, param_grid_svm, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_svm.fit(X_train_global, y_train_global)

# Ocena modelu SVM na danych treningowych i testowych
print("Ocena modelu SVM na danych treningowych:")
print(classification_report(y_train_global, grid_search_svm.predict(X_train_global)))
print("Ocena modelu SVM na danych testowych:")
print(classification_report(y_test_global, grid_search_svm.predict(X_test_global)))

Ocena modelu SVM na danych treningowych:
              precision    recall  f1-score   support

     kolacja       0.77      0.80      0.79       786
       obiad       0.85      0.85      0.85       787
   sniadanie       0.88      0.84      0.86       471

    accuracy                           0.83      2044
   macro avg       0.84      0.83      0.83      2044
weighted avg       0.83      0.83      0.83      2044

Ocena modelu SVM na danych testowych:
              precision    recall  f1-score   support

     kolacja       0.67      0.70      0.68       196
       obiad       0.79      0.77      0.78       197
   sniadanie       0.77      0.74      0.75       118

    accuracy                           0.73       511
   macro avg       0.74      0.73      0.74       511
weighted avg       0.74      0.73      0.73       511



In [ ]:
# Pipeline dla Random Forest
pipe_rf = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('rf', RandomForestClassifier())
])

# Grid search dla Random Forest
param_grid_rf = {
    'tfidf__max_df': [0.5, 0.75],
    'tfidf__min_df': [1, 2],
    'rf__n_estimators': [100, 200],
    'rf__max_depth': [None, 10, 20]
}

grid_search_rf = GridSearchCV(pipe_rf, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X_train_global, y_train_global)

# Ocena modelu Random Forest na danych treningowych i testowych
print("Ocena modelu Random Forest na danych treningowych:")
print(classification_report(y_train_global, grid_search_rf.predict(X_train_global)))
print("Ocena modelu Random Forest na danych testowych:")
print(classification_report(y_test_global, grid_search_rf.predict(X_test_global)))

Ocena modelu Random Forest na danych treningowych:
              precision    recall  f1-score   support

     kolacja       0.92      0.90      0.91       786
       obiad       0.94      0.94      0.94       787
   sniadanie       0.92      0.95      0.93       471

    accuracy                           0.93      2044
   macro avg       0.93      0.93      0.93      2044
weighted avg       0.93      0.93      0.93      2044

Ocena modelu Random Forest na danych testowych:
              precision    recall  f1-score   support

     kolacja       0.62      0.66      0.64       196
       obiad       0.75      0.76      0.76       197
   sniadanie       0.80      0.68      0.73       118

    accuracy                           0.70       511
   macro avg       0.72      0.70      0.71       511
weighted avg       0.71      0.70      0.71       511



In [ ]:
# Pipeline dla Gradient Boosting

pipe_gb = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('gb', GradientBoostingClassifier())
])

# Grid search dla Gradient Boosting
param_grid_gb = {
    'tfidf__max_df': [0.5, 0.75],
    'tfidf__min_df': [1, 2],
    'gb__n_estimators': [100, 200],
    'gb__learning_rate': [0.01, 0.1],
    'gb__max_depth': [3, 5]
}

grid_search_gb = GridSearchCV(pipe_gb, param_grid_gb, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_gb.fit(X_train_global, y_train_global)

# Ocena modelu Gradient Boosting na danych treningowych i testowych
print("Ocena modelu Gradient Boosting na danych treningowych:")
print(classification_report(y_train_global, grid_search_gb.predict(X_train_global)))
print("Ocena modelu Gradient Boosting na danych testowych:")
print(classification_report(y_test_global, grid_search_gb.predict(X_test_global)))

Ocena modelu Gradient Boosting na danych treningowych:
              precision    recall  f1-score   support

     kolacja       0.81      0.85      0.83       786
       obiad       0.89      0.88      0.88       787
   sniadanie       0.88      0.82      0.85       471

    accuracy                           0.85      2044
   macro avg       0.86      0.85      0.85      2044
weighted avg       0.86      0.85      0.85      2044

Ocena modelu Gradient Boosting na danych testowych:
              precision    recall  f1-score   support

     kolacja       0.60      0.66      0.63       196
       obiad       0.75      0.74      0.74       197
   sniadanie       0.77      0.67      0.72       118

    accuracy                           0.69       511
   macro avg       0.71      0.69      0.70       511
weighted avg       0.70      0.69      0.69       511



Wnioski z tradycyjnych modeli klasyfikacji:

Użyliśmy tu metryki accuracy do oceny klasyfiikacji. Definiowana jest jako stosunek liczby poprawnych przewidywań do całkowitej liczby przewidywań. Jest ona łatwa do zrozumienia i interpretacji.

Regresja Logistyczna
Dokładność na danych treningowych: Model regresji logistycznej osiągnął dokładność na poziomie 82%.
Dokładność na danych testowych: Dokładność na danych testowych wyniosła 73%, co sugeruje, że model ma dobrą generalizację.

SVM
Dokładność na danych treningowych: Model SVM osiągnął dokładność na poziomie 83%.
Dokładność na danych testowych: Dokładność na danych testowych wyniosła 73%, co jest zbliżone do wyników regresji logistycznej.

Random Forest
Dokładność na danych treningowych: Model Random Forest osiągnął dokładność na poziomie 93% (może to wskazywać na przeuczenie - overfitting).
Dokładność na danych testowych: Dokładność na danych testowych wyniosła 70%, co jest nieco niższe niż w przypadku innych modeli.

Gradient Boosting
Dokładność na danych treningowych: Model Gradient Boosting osiągnął dokładność na poziomie 85%.
Dokładność na danych testowych: Dokładność na danych testowych wyniosła 69%, co jest lepsze niż wyniki Random Forest, ale gorsze niż regresji logistycznej i SVM.

### Poprawa jokości najlepszego modelu SVM

Aby poprawić jakość modeli klasyfikacyjnych zastosujemy zmianę hiperparametrów w GridSearch, dodanie lematyzacji (czyli sprowadzenie słowa do podstawowej formy, poprzez obcięcie końcówki wyrazu) oraz użycie n-gramów w TfidfVectorizer

In [ ]:
# Wczytanie polskiego modelu spaCy do przetworzenia języka naturalnego w tym lematyzację.
nlp = spacy.load('pl_core_news_lg')

# Funkcja do lematyzacji
def lemmatize_text(text):
    return ' '.join([token.lemma_ for token in nlp(text)])

# Przetwarzanie kolumny z danymi
X_lemmatized = X_train_global.apply(lemmatize_text)

# Pipeline z lematyzacją, TF-IDF (zmiana tekstu na wektory) z n-gramami (unigramy-pojedyńcze słowa, bigramy-pary słów) i random forest
pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 3))),  # Użycie unigramów i bigramów
    ('svm', SVC(class_weight='balanced'))
])

# Rozszerzona przestrzeń hiperparametrów
param_grid = {
    'tfidf__max_df': [0.5, 0.75], #określają minimalną i maksymalną częstotliwość dokumentów
    'tfidf__min_df': [1, 2],
    'svm__C': [0.1, 1, 10],  # Parametr regularyzacji
    'svm__kernel': ['linear', 'rbf']  # Rodzaj funkcji jądra
}

# Cross-validation setup, dzięki temu zestw treningowy i walidacyjny ma podobny rozkład klas
cv = StratifiedKFold(n_splits=5)

# Grid search
grid_search = GridSearchCV(pipe, param_grid, cv=cv, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_lemmatized, y_train_global)

# Wyniki
print("Najlepsze parametry:", grid_search.best_params_)
print("Najlepsza dokładność:", grid_search.best_score_)
print(classification_report(y_test_global, grid_search.predict(X_test_global.apply(lemmatize_text))))

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Najlepsze parametry: {'svm__C': 1, 'svm__kernel': 'linear', 'tfidf__max_df': 0.5, 'tfidf__min_df': 2}
Najlepsza dokładność: 0.6438311999616472
              precision    recall  f1-score   support

     kolacja       0.64      0.64      0.64       196
       obiad       0.75      0.73      0.74       197
   sniadanie       0.76      0.80      0.78       118

    accuracy                           0.71       511
   macro avg       0.72      0.72      0.72       511
weighted avg       0.71      0.71      0.71       511



Po zastosowaniu lematyzacji za pomocą spacy i optymalizacji hiperparametrów dla modelu SVM uzyskaliśmy najlepsze parametry jak: rf__max_depth: None, 'svm__C': 1, 'svm__kernel': 'linear', 'tfidf__max_df': 0.5, 'tfidf__min_df': 2, a całkowitą dokładność na poziomie 0.64.
Wynika z tego że SVM okazał się być najlepszym modelem do klasyfikacji kategorii przepisów kulinarnych ponieważ uzyskał najlepsze rezultaty

## Model NLP z zatosowanie kolumny processed_ingredients

In [ ]:
# Definiuję zbiory danych niezależnych i zależnych
X = df_after_processing['processed_ingredients']
y = df_after_processing['category']

# Dzielę zbiory dane na zestaw treningowy i testowy, który odkładamy do walidacji
X_train_global, X_test_global, y_train_global, y_test_global = train_test_split(X,
                                                                                y,
                                                                                test_size=0.2,
                                                                                random_state=42,
                                                                                stratify=y) # parametr ten zapewnia proporcje klas w podziałach będą odpowiadać proporcjom w całym zbiorze danych
X = X_train_global
y = y_train_global

In [ ]:
# Definiowanie pipeline dla regresji logistycznej
pipe_lr = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('lr', LogisticRegression())
])

# Grid search dla regresji logistycznej
param_grid_lr = {
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__min_df': [1, 2, 3],
    'lr__C': [0.01, 0.1, 1, 10, 100]
}

grid_search_lr = GridSearchCV(pipe_lr, param_grid_lr, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_lr.fit(X_train_global, y_train_global)

# Ocena modelu na danych treningowych
print("Ocena modelu regresji logistycznej na danych treningowych:")
print(classification_report(y_train_global, grid_search_lr.predict(X_train_global)))

# Ocena modelu na danych testowych
print("Ocena modelu regresji logistycznej na danych testowych:")
print(classification_report(y_test_global, grid_search_lr.predict(X_test_global)))

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Ocena modelu regresji logistycznej na danych treningowych:
              precision    recall  f1-score   support

     kolacja       0.77      0.77      0.77       786
       obiad       0.82      0.86      0.84       787
   sniadanie       0.86      0.79      0.82       471

    accuracy                           0.81      2044
   macro avg       0.82      0.81      0.81      2044
weighted avg       0.81      0.81      0.81      2044

Ocena modelu regresji logistycznej na danych testowych:
              precision    recall  f1-score   support

     kolacja       0.69      0.72      0.71       196
       obiad       0.79      0.82      0.80       197
   sniadanie       0.80      0.69      0.74       118

    accuracy                           0.75       511
   macro avg       0.76      0.74      0.75       511
weighted avg       0.75      0.75      0.75       511



In [ ]:
# Pipeline dla SVM
pipe_svm = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('svm', SVC())
])

# Grid search dla SVM
param_grid_svm = {
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__min_df': [1, 2, 3],
    'svm__C': [0.1, 1, 10],
    'svm__kernel': ['linear', 'rbf']
}

grid_search_svm = GridSearchCV(pipe_svm, param_grid_svm, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_svm.fit(X_train_global, y_train_global)

# Ocena modelu SVM na danych treningowych i testowych
print("Ocena modelu SVM na danych treningowych:")
print(classification_report(y_train_global, grid_search_svm.predict(X_train_global)))
print("Ocena modelu SVM na danych testowych:")
print(classification_report(y_test_global, grid_search_svm.predict(X_test_global)))

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Ocena modelu SVM na danych treningowych:
              precision    recall  f1-score   support

     kolacja       0.77      0.81      0.79       786
       obiad       0.85      0.85      0.85       787
   sniadanie       0.87      0.82      0.84       471

    accuracy                           0.83      2044
   macro avg       0.83      0.82      0.83      2044
weighted avg       0.83      0.83      0.83      2044

Ocena modelu SVM na danych testowych:
              precision    recall  f1-score   support

     kolacja       0.69      0.71      0.70       196
       obiad       0.80      0.79      0.80       197
   sniadanie       0.78      0.74      0.76       118

    accuracy                           0.75       511
   macro avg       0.75      0.75      0.75       511
weighted avg       0.75      0.75      0.75       511



In [ ]:
# Pipeline dla Random Forest
pipe_rf = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('rf', RandomForestClassifier())
])

# Grid search dla Random Forest
param_grid_rf = {
    'tfidf__max_df': [0.5, 0.75],
    'tfidf__min_df': [1, 2],
    'rf__n_estimators': [100, 200],
    'rf__max_depth': [None, 10, 20]
}

grid_search_rf = GridSearchCV(pipe_rf, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X_train_global, y_train_global)

# Ocena modelu Random Forest na danych treningowych
print("Ocena modelu Random Forest na danych treningowych:")
print(classification_report(y_train_global, grid_search_rf.predict(X_train_global)))

# Ocena modelu Random Forest na danych testowych
print("Ocena modelu Random Forest na danych testowych:")
print(classification_report(y_test_global, grid_search_rf.predict(X_test_global)))

Ocena modelu Random Forest na danych treningowych:
              precision    recall  f1-score   support

     kolacja       0.71      0.87      0.78       786
       obiad       0.88      0.84      0.86       787
   sniadanie       0.91      0.63      0.74       471

    accuracy                           0.80      2044
   macro avg       0.83      0.78      0.80      2044
weighted avg       0.82      0.80      0.80      2044

Ocena modelu Random Forest na danych testowych:
              precision    recall  f1-score   support

     kolacja       0.57      0.70      0.63       196
       obiad       0.75      0.76      0.76       197
   sniadanie       0.81      0.49      0.61       118

    accuracy                           0.68       511
   macro avg       0.71      0.65      0.67       511
weighted avg       0.70      0.68      0.68       511



In [ ]:
# Pipeline dla Gradient Boosting

pipe_gb = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('gb', GradientBoostingClassifier())
])

# Grid search dla Gradient Boosting
param_grid_gb = {
    'tfidf__max_df': [0.5, 0.75],
    'tfidf__min_df': [1, 2],
    'gb__n_estimators': [100, 200],
    'gb__learning_rate': [0.01, 0.1],
    'gb__max_depth': [3, 5]
}

grid_search_gb = GridSearchCV(pipe_gb, param_grid_gb, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_gb.fit(X_train_global, y_train_global)

# Ocena modelu Gradient Boosting na danych treningowych
print("Ocena modelu Gradient Boosting na danych treningowych:")
print(classification_report(y_train_global, grid_search_gb.predict(X_train_global)))

# Ocena modelu Gradient Boosting na danych testowych
print("Ocena modelu Gradient Boosting na danych testowych:")
print(classification_report(y_test_global, grid_search_gb.predict(X_test_global)))

Ocena modelu Gradient Boosting na danych treningowych:
              precision    recall  f1-score   support

     kolacja       0.81      0.85      0.83       786
       obiad       0.88      0.89      0.89       787
   sniadanie       0.88      0.81      0.85       471

    accuracy                           0.85      2044
   macro avg       0.86      0.85      0.85      2044
weighted avg       0.86      0.85      0.85      2044

Ocena modelu Gradient Boosting na danych testowych:
              precision    recall  f1-score   support

     kolacja       0.62      0.68      0.65       196
       obiad       0.77      0.75      0.76       197
   sniadanie       0.74      0.66      0.70       118

    accuracy                           0.70       511
   macro avg       0.71      0.70      0.70       511
weighted avg       0.71      0.70      0.70       511



Wnioski, po zastosowaniu kolumny processed_ingredients wyniki nieznacznie poprawiły się na danych testowych przy modelach takich jak Regresja Liniowa czy Svm oraz GradienBoosting, a nie znaacznie pogorszyły na modelu Radnom Forest, dlatego do dalszych obliczeń wykorzystamy tą kolumne.

## Spacy z zastosowaniem kolumny processed_ingredients

SpaCy to popularna biblioteka NLP, która oferuje zaawansowane funkcje przetwarzania języka naturalnego, w tym lematyzację. Posiada modele dla wielu języków, w tym polskiego (pl_core_news_lg).

In [ ]:
# Wczytujemy model jezykowy dla języka polskiego
nlp = spacy.load('pl_core_news_lg')

In [ ]:
# Tworzymy funkcję do przetwarzania tekstu, funkcja tokenize przyjmuje zdania i przetwarza za pomocą Spacy a następnie zwraca listę tokenów (słów)
def tokenize(sent):
  doc = nlp(sent)
  return [token.text for token in doc]

In [ ]:
# Przetwarzamy dane tektowe z kolumny processed_ingredients aby uzuskać listę tokenów
tokens = tokenize(' '.join(df_after_processing['processed_ingredients']).lower())

In [ ]:
# Funkcja lemmatize przyjmuje zdanie przetwarza je oraz zwraca tekst lematyzowanymi słowami
def lemmatize(sent):
  doc = nlp(sent)
  return ' '.join([token.lemma_ for token in doc])

In [ ]:
# Lematyzujemy tekst w kolumnie ingredients i zapisujemy wyniki do nowej kolumny ingredients_proc
df_after_processing['ingredients_proc'] = df_after_processing['processed_ingredients'].map(lemmatize)
df_after_processing['ingredients_proc'] = df_after_processing['processed_ingredients'].map(lambda sent: ' '.join([token.lemma_ for token in nlp(sent)]))

In [ ]:
# Funkcja clear_teks usuwa stopwordsy i cyfry a nazstępnie zwraca lematyzowany tekst
def clear_text(sent):
  return ' '.join([token.lemma_ for token in nlp(sent) if not token.is_stop and not token.is_digit])

In [ ]:
# Przetwarzamy teksy w kolumnie ingredients usuwając stopwordsy i cyfry i zapisujemy wynik w kolumnie ingredients_proc
df_after_processing['ingredients_proc'] = df_after_processing['processed_ingredients'].map(clear_text)

In [ ]:
df_after_processing.head()

,category,recipe_title,ingredients,preparation_method,rating,opinions_count,preparation_time,vegetarian,processed_ingredients,ingredients_proc
0,sniadanie,Jajka z rzeżuchą,10 jajek;4 łyżki majonezu;2 łyżeczki musztardy...,Jajka ugotować na twardo (6 minut od zagotowan...,NaN,NaN,NaN,<NA>,"jajek, majonezu, musztardy miodowej, sosu worc...","jajko , majonez , musztarda miodowy , sos worc..."
1,sniadanie,Pasta z fasoli,1 puszka białej fasolki (400 g);150 g korzenia...,Fasolkę odcedzić na sitku. Selera obrać i pokr...,NaN,NaN,NaN,<NA>,"puszka białej fasolki ( g), korzenia selera, p...","puszka biały fasolka ( gram ) , korzenie seler..."
2,sniadanie,Jajka z majonezem truflowym,5 jajek;2 łyżki majonezu;ok. 2 łyżeczek pasty ...,"Jajka ugotować na twardo, obrać ze skorupek, p...",NaN,NaN,NaN,<NA>,"jajek, majonezu, ok pasty salsy truflowej, nat...","jajko , majonez , pasta salsy truflowy , natka..."
3,sniadanie,Pan bagnat,"1 większa, okrągła lub podłużna bułka, np. cia...",Pieczywo przekroić w poziomie. Położyć na blac...,5.0,4.0,NaN,<NA>,"większa, okrągła podłużna bułka, np ciabatta, ...","duży , okrąc podłużny bułka , na przykład ciab..."
4,sniadanie,Tosty francuskie z solonym karmelem,4 kromki brioche lub chałki;1 duże jajko;5 łyż...,W głębokim talerzu roztrzepać (widelcem lub ró...,5.0,2.0,NaN,<NA>,"kromki brioche chałki, duże jajko, mleka, cukr...","kromka brioche chałka , duży jajko , mleko , c..."


In [ ]:
# Przygotowanie danych do trenowania modeli
X = df_after_processing['ingredients_proc']
y = df_after_processing['category']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y) # zachowujemy proporcję klas

In [ ]:
# Modelowanie TF-IDF i regresja logistyczna
tfidf = TfidfVectorizer(max_features=100) # konwertuje tekst z max 100 cechami
tfidf.fit(X_train) # dopasowujemy TfidfVectorizer do danych treningowych
X_train_tf = tfidf.transform(X_train) # transformujemy dane treningowe na wektory TF-IDF
lr = LogisticRegression() # trenujemy model regresji logistycznej na wektorach TF-IDF
lr.fit(X_train_tf, y_train)

LogisticRegression()

In [ ]:
# Ocena modelu na danych treningowych
print(classification_report(y_train, lr.predict(X_train_tf)))

              precision    recall  f1-score   support

     kolacja       0.62      0.64      0.63       786
       obiad       0.72      0.74      0.73       787
   sniadanie       0.73      0.66      0.70       471

    accuracy                           0.68      2044
   macro avg       0.69      0.68      0.69      2044
weighted avg       0.68      0.68      0.68      2044



In [ ]:
# Funkcja to_pos prztwarza zdanie za pomocą Spacy i zwraca tekst z częściami mowy dla każdego tokena
def to_pos(sent):
  doc = nlp(sent)
  return ' '.join([token.pos_ for token in doc])

In [ ]:
# Przetwarzamy tekst w kolumnie processed_ingredients na części mowy i zapisujmey w nowej kolumnie pos
df_after_processing['pos'] = df_after_processing['processed_ingredients'].map(to_pos)

In [ ]:
df_after_processing.head()

,category,recipe_title,ingredients,preparation_method,rating,opinions_count,preparation_time,vegetarian,processed_ingredients,ingredients_proc,pos
0,sniadanie,Jajka z rzeżuchą,10 jajek;4 łyżki majonezu;2 łyżeczki musztardy...,Jajka ugotować na twardo (6 minut od zagotowan...,NaN,NaN,NaN,<NA>,"jajek, majonezu, musztardy miodowej, sosu worc...","jajko , majonez , musztarda miodowy , sos worc...",NOUN PUNCT NOUN PUNCT NOUN ADJ PUNCT NOUN VERB...
1,sniadanie,Pasta z fasoli,1 puszka białej fasolki (400 g);150 g korzenia...,Fasolkę odcedzić na sitku. Selera obrać i pokr...,NaN,NaN,NaN,<NA>,"puszka białej fasolki ( g), korzenia selera, p...","puszka biały fasolka ( gram ) , korzenie seler...",NOUN ADJ NOUN PUNCT X PUNCT PUNCT NOUN NOUN PU...
2,sniadanie,Jajka z majonezem truflowym,5 jajek;2 łyżki majonezu;ok. 2 łyżeczek pasty ...,"Jajka ugotować na twardo, obrać ze skorupek, p...",NaN,NaN,NaN,<NA>,"jajek, majonezu, ok pasty salsy truflowej, nat...","jajko , majonez , pasta salsy truflowy , natka...",NOUN PUNCT NOUN PUNCT ADP NOUN NOUN ADJ PUNCT ...
3,sniadanie,Pan bagnat,"1 większa, okrągła lub podłużna bułka, np. cia...",Pieczywo przekroić w poziomie. Położyć na blac...,5.0,4.0,NaN,<NA>,"większa, okrągła podłużna bułka, np ciabatta, ...","duży , okrąc podłużny bułka , na przykład ciab...",ADJ PUNCT ADJ ADJ NOUN PUNCT X PROPN PUNCT NOU...
4,sniadanie,Tosty francuskie z solonym karmelem,4 kromki brioche lub chałki;1 duże jajko;5 łyż...,W głębokim talerzu roztrzepać (widelcem lub ró...,5.0,2.0,NaN,<NA>,"kromki brioche chałki, duże jajko, mleka, cukr...","kromka brioche chałka , duży jajko , mleko , c...",NOUN NOUN NOUN PUNCT ADJ NOUN PUNCT NOUN PUNCT...


In [ ]:
# Przygotowanie danych do trenowania modeli
X = df_after_processing['pos']
y = df_after_processing['category']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y)

In [ ]:
# Modelowanie TF-IDF i regresja logistyczna
tfidf = TfidfVectorizer(max_features=1000, ngram_range=(1,3)) # konwertujemy tekst na wektory TF-IDF z maksymalnie 1000 cechami, używając unigramów, bigramów i trigramów
tfidf.fit(X_train)
X_train_tf = tfidf.transform(X_train)
lr = LogisticRegression()
lr.fit(X_train_tf, y_train)

LogisticRegression()

In [ ]:
# Ocena modelu na danych treningowych
print(classification_report(y_train, lr.predict(X_train_tf)))

              precision    recall  f1-score   support

     kolacja       0.52      0.58      0.55       786
       obiad       0.57      0.67      0.62       787
   sniadanie       0.60      0.32      0.41       471

    accuracy                           0.55      2044
   macro avg       0.56      0.52      0.53      2044
weighted avg       0.56      0.55      0.54      2044



Wnioski:
Przy porównaniu tych dwóch metod lematyzacja wydaje się być bardziej skuteczna niż pos tagging. Model regresji logistycznej przy lematyzacji osiągnął 68% dokładności zaś przy zastosowaniu pos taggingu tylko 55% dokładności. Dla poprawy skuteczności modeli mozna było by zastosować bardziej zaawansowane techniki NLP takie jak embeddings (np. Word2Vec albo Bert) oraz optymalizację hiperparametrów jak również eksploracja innych modeli klasyfikacyjnych.

## Model głębokiej sieci neuronowej (DNN)

model ten stosujemy do klasyfikacji rodzajów posiłków na podstawie składników, używając biblioteki TensorFlow i Keras.

In [ ]:
# Inicjalizacja Tokenizera z limitem 10000 najcześciej występujących słów
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>") # jak traktowane są słowa, tu każde słowo które nie zostało zobaczone podczas fit_on_tekst zostanie zastąpiąne tokenem OOV (out of vocabulary) podczas texts_to_sequence
tokenizer.fit_on_texts(X_train_global)

# Zamiana tekstów na sekwencje liczb, gdzie każde słowo jest reprezentowane przez unikalny indeks
train_sequences = tokenizer.texts_to_sequences(X_train_global)
test_sequences = tokenizer.texts_to_sequences(X_test_global)

# Padding sekwencji do tej samej długości, dodanie wypełnienia padding lub przycięcia truncating aby wszytskie miały długość 120 tokenów
train_padded = pad_sequences(train_sequences, maxlen=120, truncating='post', padding='post')
test_padded = pad_sequences(test_sequences, maxlen=120, truncating='post', padding='post')

In [ ]:
# Kodowanie etykiet jako liczby całkowite
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_global)
y_test_encoded = label_encoder.transform(y_test_global)

# One-hot encoding etykiet, przekształcająć je w macierze binarne
y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

In [ ]:
# Definiowanie modelu_1 składający się z warstwy embeddingu oraz GlobalAveragePooling1D i dwóch warst dense
model_1 = Sequential([
    Embedding(10000, 16, input_length=120),
    GlobalAveragePooling1D(),
    #Dense(24, activation='relu'),
    Dense(12, activation='relu'),
    Dense(6, activation='sigmoid'),
    #Dense(1, activation=None),
    Dense(y_train_categorical.shape[1], activation='softmax')
])

# Kompilacja modelu
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Podsumowanie modelu
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 12)                204       
                                                                 
 dense_1 (Dense)             (None, 6)                 78        
                                                                 
 dense_2 (Dense)             (None, 3)                 21        
                                                                 
Total params: 160303 (626.18 KB)
Trainable params: 160303 (626.18 KB)
Non-trainable params: 0 (0.00 Byte)
________________

In [ ]:
# Trenowanie modelu
history_1 = model_1.fit(train_padded, y_train_categorical, epochs=100, validation_data=(test_padded, y_test_categorical))

Epoch 1/100
64/64 [==============================] - 3s 14ms/step - loss: 1.3970 - accuracy: 0.3845 - val_loss: 1.2836 - val_accuracy: 0.3836
Epoch 2/100
64/64 [==============================] - 0s 6ms/step - loss: 1.1985 - accuracy: 0.3845 - val_loss: 1.1308 - val_accuracy: 0.3836
Epoch 3/100
64/64 [==============================] - 0s 6ms/step - loss: 1.0990 - accuracy: 0.3845 - val_loss: 1.0818 - val_accuracy: 0.3836
Epoch 4/100
64/64 [==============================] - 0s 6ms/step - loss: 1.0744 - accuracy: 0.3845 - val_loss: 1.0717 - val_accuracy: 0.3836
Epoch 5/100
64/64 [==============================] - 0s 7ms/step - loss: 1.0692 - accuracy: 0.3845 - val_loss: 1.0690 - val_accuracy: 0.3836
Epoch 6/100
64/64 [==============================] - 0s 6ms/step - loss: 1.0663 - accuracy: 0.3850 - val_loss: 1.0661 - val_accuracy: 0.4188
Epoch 7/100
64/64 [==============================] - 0s 6ms/step - loss: 1.0628 - accuracy: 0.5049 - val_loss: 1.0622 - val_accuracy: 0.5068
Epoch 8/100


In [ ]:
# Ocena modelu na danych testowych
test_loss, test_acc = model_1.evaluate(test_padded, y_test_categorical)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

16/16 [==============================] - 0s 3ms/step - loss: 1.0288 - accuracy: 0.7045
Test Loss: 1.0287959575653076
Test Accuracy: 0.7045009732246399


In [ ]:
# Przygotowanie nowego tekstu do predykcji
new_texts = [
    "jajka, mąka, mleko, cukier, masło", # naleśniki
    "pomidor, mozzarella, bazylia, oliwa z oliwek", # sałtka caprese
    "kurczak, ryż, curry, mleko kokosowe"  # danie z kurczakiem
]
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_padded = pad_sequences(new_sequences, maxlen=120, padding='post', truncating='post')

# Wykonanie predykcji
predictions = model_1.predict(new_padded)
predicted_classes = np.argmax(predictions, axis=1)

# Przekształcenie ideksów klas z powrtotem na etykiety
predicted_labels = label_encoder.inverse_transform(predicted_classes)
print(predicted_labels)

1/1 [==============================] - 0s 21ms/step
['sniadanie' 'sniadanie' 'kolacja']


In [ ]:
# Definiowanie modelu_2, model z warstwą SimpleRNN
model_2 = Sequential([
    Embedding(10000, 32, input_length=120),
    SimpleRNN(16),  # Dodanie warstwy SimpleRNN
    Dense(12, activation='relu'),
    Dense(y_train_categorical.shape[1], activation='softmax')
])

model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 120, 32)           320000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 16)                784       
                                                                 
 dense_3 (Dense)             (None, 12)                204       
                                                                 
 dense_4 (Dense)             (None, 3)                 39        
                                                                 
Total params: 321027 (1.22 MB)
Trainable params: 321027 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Trenowanie modelu
history_2 = model_2.fit(train_padded, y_train_categorical, epochs=100, validation_data=(test_padded, y_test_categorical))

Epoch 1/100
64/64 [==============================] - 5s 54ms/step - loss: 1.0771 - accuracy: 0.3787 - val_loss: 1.0759 - val_accuracy: 0.3855
Epoch 2/100
64/64 [==============================] - 4s 57ms/step - loss: 1.0580 - accuracy: 0.4667 - val_loss: 1.0767 - val_accuracy: 0.4227
Epoch 3/100
64/64 [==============================] - 2s 32ms/step - loss: 1.0263 - accuracy: 0.5157 - val_loss: 1.1001 - val_accuracy: 0.4227
Epoch 4/100
64/64 [==============================] - 2s 30ms/step - loss: 0.9731 - accuracy: 0.5592 - val_loss: 1.1273 - val_accuracy: 0.4227
Epoch 5/100
64/64 [==============================] - 2s 31ms/step - loss: 0.9014 - accuracy: 0.6052 - val_loss: 1.1968 - val_accuracy: 0.4031
Epoch 6/100
64/64 [==============================] - 2s 32ms/step - loss: 0.8425 - accuracy: 0.6566 - val_loss: 1.2133 - val_accuracy: 0.4266
Epoch 7/100
64/64 [==============================] - 2s 33ms/step - loss: 0.8094 - accuracy: 0.6751 - val_loss: 1.2264 - val_accuracy: 0.4462
Epoch 

In [ ]:
# Ocena modelu na danych testowych
test_loss, test_acc = model_2.evaluate(test_padded, y_test_categorical)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

16/16 [==============================] - 0s 9ms/step - loss: 2.3133 - accuracy: 0.3738
Test Loss: 2.3133225440979004
Test Accuracy: 0.373776912689209


In [ ]:
# Przygotowanie nowego tekstu do predykcji
new_texts = [
    "jajka, mąka, mleko, cukier, masło", # naleśniki
    "pomidor, mozzarella, bazylia, oliwa z oliwek", # sałtka caprese
    "kurczak, ryż, curry, mleko kokosowe"  # danie z kurczakiem
]
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_padded = pad_sequences(new_sequences, maxlen=120, padding='post', truncating='post')

# Wykonanie predykcji
predictions = model_2.predict(new_padded)
predicted_classes = np.argmax(predictions, axis=1)

# Przekształcenie ideksów klas z powrtotem na etykiety
predicted_labels = label_encoder.inverse_transform(predicted_classes)
print(predicted_labels)

1/1 [==============================] - 0s 187ms/step
['sniadanie' 'kolacja' 'sniadanie']


In [ ]:
# Definiowanie modelu_3, model z warstwą Bidirectional LSTM
model_3 = Sequential([
    Embedding(10000, 32, input_length=120),
    Bidirectional(LSTM(16)),  # Dodanie warstwy Bidirectional LSTM
    Dense(12, activation='relu'),
    Dense(y_train_categorical.shape[1], activation='softmax')
])

model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 120, 32)           320000    
                                                                 
 bidirectional (Bidirection  (None, 32)                6272      
 al)                                                             
                                                                 
 dense_5 (Dense)             (None, 12)                396       
                                                                 
 dense_6 (Dense)             (None, 3)                 39        
                                                                 
Total params: 326707 (1.25 MB)
Trainable params: 326707 (1.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Trenowanie modelu
history_3 = model_3.fit(train_padded, y_train_categorical, epochs=100, validation_data=(test_padded, y_test_categorical))

Epoch 1/100
64/64 [==============================] - 11s 93ms/step - loss: 1.0182 - accuracy: 0.4237 - val_loss: 0.8661 - val_accuracy: 0.5890
Epoch 2/100
64/64 [==============================] - 8s 123ms/step - loss: 0.7607 - accuracy: 0.6566 - val_loss: 0.7056 - val_accuracy: 0.6947
Epoch 3/100
64/64 [==============================] - 5s 80ms/step - loss: 0.5996 - accuracy: 0.7441 - val_loss: 0.6952 - val_accuracy: 0.7006
Epoch 4/100
64/64 [==============================] - 10s 152ms/step - loss: 0.5144 - accuracy: 0.7803 - val_loss: 0.7176 - val_accuracy: 0.6967
Epoch 5/100
64/64 [==============================] - 5s 81ms/step - loss: 0.4645 - accuracy: 0.8112 - val_loss: 0.7314 - val_accuracy: 0.6928
Epoch 6/100
64/64 [==============================] - 5s 79ms/step - loss: 0.4130 - accuracy: 0.8346 - val_loss: 0.7792 - val_accuracy: 0.6849
Epoch 7/100
64/64 [==============================] - 8s 124ms/step - loss: 0.3772 - accuracy: 0.8415 - val_loss: 0.8178 - val_accuracy: 0.6751
E

In [ ]:
# Ocena modelu na danych testowych
test_loss, test_acc = model_3.evaluate(test_padded, y_test_categorical)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

16/16 [==============================] - 0s 16ms/step - loss: 2.0334 - accuracy: 0.6399
Test Loss: 2.0333573818206787
Test Accuracy: 0.6399217247962952


In [ ]:
# Przygotowanie nowego tekstu do predykcji
new_texts = [
    "jajka, mąka, mleko, cukier, masło", # naleśniki
    "pomidor, mozzarella, bazylia, oliwa z oliwek", # sałtka caprese
    "kurczak, ryż, curry, mleko kokosowe"  # danie z kurczakiem
]
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_padded = pad_sequences(new_sequences, maxlen=120, padding='post', truncating='post')

# Wykonanie predykcji
predictions = model_3.predict(new_padded)
predicted_classes = np.argmax(predictions, axis=1)

# Przekształcenie ideksów klas z powrotem na etykiety
predicted_labels = label_encoder.inverse_transform(predicted_classes)
print(predicted_labels)

1/1 [==============================] - 2s 2s/step
['sniadanie' 'sniadanie' 'obiad']


Wnioski dla modeli głębokich sieci neuronowych:

Prosta sieć neuronowa (model_1) osiągnął
najwyższą dokładność (70.45%) spośród trzech testowanych modeli, co sugeruje, że jest on najbardziej skuteczny w klasyfikacji danych testowych. Użycie warstwy GlobalAveragePooling1D pozwala na redukcję wymiarów, co może pomóc w uproszczeniu modelu i zwiększeniu jego zdolności generalizacji.

SimpleRNN (model_2)
osiągnął najniższą dokładność (37.38%), co wskazuje, że jest najmniej skuteczny w klasyfikacji danych testowych.
Wysoka wartość straty sugeruje, że model może mieć problemy z zbieżnością lub jest zbyt prosty, aby dobrze uchwycić zależności w danych. SimpleRNN może mieć trudności z uchwyceniem długoterminowych zależności w sekwencjach danych tekstowych, co może prowadzić do gorszych wyników w porównaniu do innych architektur.

Bidirectional LSTM (model_3)
osiągnął umiarkowaną dokładność (63.99%), co jest lepsze niż Model 2, ale gorsze niż Model 1. Użycie warstwy Bidirectional LSTM pozwala na lepsze uchwycenie zależności w sekwencjach danych, ale może również prowadzić do problemów z przetrenowaniem, jeśli model jest zbyt skomplikowany w porównaniu do ilości danych. Bidirectional LSTM może lepiej radzić sobie z długoterminowymi zależnościami w danych tekstowych, co może wyjaśniać lepsze wyniki w porównaniu do SimpleRNN, ale nadal nie dorównuje prostszemu modelowi z GlobalAveragePooling1D.

# Wnioski końcowe
Udało się zebrać i zorganizować dużą ilość danych (2563 przepisy), które są podstawą do dalszej analizy. Następnie przeszlismy do analizy danych dzięki czemu możemy stwierdzić że przepisy są równomiernie rozłożone między kategorie (śniadania, obiady, kolacje). Usuneliśmy stopwordsy i przeprowadziliśmy standaryzacje jednostek co poprawiło jakość danych tekstowych, co było kluczowe dla dalszego modelowania. Użycie grid search i cross-validation pozwoliło na stworzenie optymalnych modeli, które dobrze radzą sobie z danymi testowymi. Po wytrenowaniu modeli możemy stwierdzić, że najlepszym modelem okazał się model Regresji Logistycznej i SVM zaś modele Random Forest i Gradient Boosting miały tendencję do przetrenowania. Zastosowanie lematyzacji poprawiło jakość danych tekstowych, co przełożyło się na lepsze wyniki modeli klasyfikacyjnych. Zaś przy modelach sieci neuronowych model_1 osiągnął najlepsze wyniki spośród testowanych modeli. Projekt wykazał, że proste modele klasyfikacyjne, takie jak regresja logistyczna i SVM, mogą być bardzo skuteczne w zadaniu klasyfikacji przepisów kulinarnych na podstawie składników.